## 학습 목표 

- 머신러닝을 위해서, numpy와 pandas의 사용법을 다시 연습합니다.

- numpy를 사용하여 여러가지 벡터 연산을 연습하고, 실제 데이터 유사도를 계산해봅시다.

### 1. 라이브러리 연습 예제 

[ 필수 RECAP ]

> Numpy : broadcasting, universal function

> Pandas : fancy indexing, categorical variable encoding

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
# data를 불러와 봅니다.
base_path='/content/drive/MyDrive/Colab Notebooks/data/prudential-life-insurance-assessment/'
sample_submission = pd.read_csv(base_path + "sample_submission.csv")
test = pd.read_csv(base_path + "test.csv")
data = pd.read_csv(base_path + "train.csv")

### 2. 유사도 측정 예제

#### EDA & Preprocessing

- 퀵하게 데이터를 파악하고, 유사도 측정 예시에 알맞게 가공해봅시다.

In [58]:
## EDA
# data.iloc[row에 대한 조건, col에 대한 조건] : 정수 index # list, tuple, numpy array
# 앞에서부터 row 100개, col 15개를 뽑아서 가져온다
# data = data.iloc[:100, :15]
# 앞에서부터 col 15개를 가져옵니다
# data = data.iloc[:, :15]

# boolean mask
# Q. 데이터 중에 Ins_Age가 0.5를 넘는 모든 사람을 뽑고 싶습니다
# data.loc[row에 대한 조건, col에 대한 조건]
# 1. 데이터 중에 Ins_Age가 0.5를 넘는
# data[mask] loc 안써도 가능
# data.loc[mask, col] 써야한다
#data.loc[data.Ins_Age >= 0.5, 'Ins_Age']
#data.loc[data.Ins_Age >= 0.5, data.columns.str.startswith('Medical_Keyword')
#data.loc[data.Ins_Age >= 0.5, ['Ht', 'Wt', 'BMI']]
index = data.loc[data.Ins_Age >= 0.5, data.columns.str.startswith('Medical_Keyword')].index
data.loc[index, 'Id']
## col 조건
# Q. Medical_Keyword라고 시작하는 모든 column을 뽑고 싶습니다.
#data.columns[data.columns.str.startswith('Medical_Keyword')]
#data.columns.str.startswith('Medical_Keyword')

0            2
5           10
7           14
8           15
9           16
         ...  
59365    79131
59366    79132
59369    79135
59371    79137
59379    79145
Name: Id, Length: 21087, dtype: int64

In [35]:
#for col in data.loc[:, data.columns.str.findall('Medical_Keyword')].columns:
#  print(data[col].unique())
#col = 'Medical Keyword'

In [100]:
data = data.iloc [:, :15]
data

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,79142,1,D1,10,0.230769,2,3,1,0.074627,0.709091,0.320084,0.519103,0.020,1,3
59377,79143,1,D3,26,0.230769,2,3,1,0.432836,0.800000,0.403766,0.551119,0.100,9,1
59378,79144,1,E1,26,0.076923,2,3,1,0.104478,0.745455,0.246862,0.360969,0.035,9,1
59379,79145,1,D2,10,0.230769,2,3,1,0.507463,0.690909,0.276151,0.462452,0.038,9,1


In [101]:
# Id column을 drop하고, Product_Info_2 column을 숫자로 변경
# 1. Id drop
# data.drop(index = ) data.drop(columns = )
# data.drop(columns='Id', inplace=True) # 결과 출력x, 원본 증발, 속도이득 없음
#df = df.drop('column_name', 1)
#where 1 is the axis number (0 for rows and 1 for columns.)
# data = data.drop(columns='Id') # 이 방법 추천
data = data.drop(columns='Id')

# 2. Product_Info_2를 숫자로 변경. Ordinal Encoding, One-Hot Encoding
# 선호도 -> ordinal Encoding
# 성별 선택 -> One-Hot Encoding
# 2-1 Ordinal Encoding (pandas)
data.Product_Info_2 = pd.factorize(data.Product_Info_2)[0]
data
# 2-2 One-hot Encoding(pandas)
#pd.get_dummies(data=data, columns=['Product_Info_2'])

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3
0,1,0,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1
1,1,1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3
2,1,2,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1
3,1,3,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1
4,1,4,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,1,7,10,0.230769,2,3,1,0.074627,0.709091,0.320084,0.519103,0.020,1,3
59377,1,0,26,0.230769,2,3,1,0.432836,0.800000,0.403766,0.551119,0.100,9,1
59378,1,2,26,0.076923,2,3,1,0.104478,0.745455,0.246862,0.360969,0.035,9,1
59379,1,4,10,0.230769,2,3,1,0.507463,0.690909,0.276151,0.462452,0.038,9,1


#### 유사도(Proximity)

1. Euclidean distance

2. Cosine Similarity

In [51]:
v1 = np.array([1,2,3])
v2 = np.array([0,0,0])

In [ ]:
L = [1,2,3]
L2 = [0,0,0]
for i, i2 in zip(L,L2):
  print(i+i2)
L+L2 # concatenation

1
2
3


[1, 2, 3, 0, 0, 0]

In [ ]:
# numpy broadcasting
v1 + v2 # vector addition 
v2 - v1 # vector subtraction
v1 * v2 # elementwise multiplication
v2 / v1 # elementwise division
v1 * 2 # scalar multiplication

## universal function
def reverse_num(x):
  return 1/x

# broadcasting
for x in v1:
  print(reverse_num(x))

1 / v1

1.0
0.5
0.3333333333333333


array([1.        , 0.5       , 0.33333333])

In [ ]:
## v1과 v2에 있는 모든 원소들끼리 뺀 다음에, 제곱하고 다 더해서 루트 씌운것 = Euclidean distance(L2 distance)
(v1 - v2) ** 2
distance = np.sqrt(np.sum(np.square(v1-v2)))
distance

distance2 =np.linalg.norm(v1-v2) # 벡터의 크기(L2 norm)

distance == distance2

True

In [ ]:
f = lambda x,y : np.sqrt(np.sum(np.square(x-y)))
f(v1,v2)

3.7416573867739413

In [97]:
def euclidean_distance(a,b):
    distance = np.linalg.norm(a-b)
    return distance
#euclidean_distance(v1,v2)

In [98]:
def cosine_similarity(a,b):
    # 1. 분자 : a o b a와 b의 내적
    dot = a @ b
    # 2. 분모 : |a|, |b|
    norm = np.linalg.norm(a) * np.linalg.norm(b) + 1e-12
    return dot / norm

In [54]:
cosine_similarity(v1,v2)

0.0

### 6번 고객과 가장 가까운 고객 찾기

In [66]:
def Euclidean_Dist(df1, df2, cols=['x_coord','y_coord']):
    return np.linalg.norm(df1[cols].values - df2[cols].values,
                   axis=1)

In [57]:
target = data.loc[2]

Id                           6
Product_Info_1               1
Product_Info_2              E1
Product_Info_3              26
Product_Info_4        0.076923
                        ...   
Medical_Keyword_45           0
Medical_Keyword_46           0
Medical_Keyword_47           0
Medical_Keyword_48           0
Response                     8
Name: 2, Length: 128, dtype: object

In [64]:
data

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3
0,1,0,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1
1,1,1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3
2,1,2,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1
3,1,3,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1
4,1,4,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,1,7,10,0.230769,2,3,1,0.074627,0.709091,0.320084,0.519103,0.020,1,3
59377,1,0,26,0.230769,2,3,1,0.432836,0.800000,0.403766,0.551119,0.100,9,1
59378,1,2,26,0.076923,2,3,1,0.104478,0.745455,0.246862,0.360969,0.035,9,1
59379,1,4,10,0.230769,2,3,1,0.507463,0.690909,0.276151,0.462452,0.038,9,1


In [115]:
def euclidean_distance(a,b):
    return np.linalg.norm(a-b)

In [105]:
def euclidean_distance(a,b):
    return np.sqrt(np.sum(np.square(a-b)))

In [114]:
def cosine_similarity(a,b):
    # 1. 분자 : a o b a와 b의 내적
    dot = a @ b
    # 2. 분모 : |a|, |b|
    norm = np.linalg.norm(a) * np.linalg.norm(b) + 1e-12
    return dot / norm

In [116]:
## target을 기준으로 모든 데이터에 대해서 유사도를 계산한뒤, 가장 유사한 고객을 찾아주세요. (index 출력)
# euclidean -> cosine
target = data.loc[2]

# for i in data.iterrows():
#   print(i)
#---------------------------------------------
# for idx, row in data.iterrows():
#   if idx < 5:
#     print(euclidean_distance(target,row))
#   else:
#     break
distances = []
for idx, row in data.iterrows():
  distances.append(euclidean_distance(target, row))

In [ ]:
for idx, row in data.iterrows():
  if idx == 2:
    continue
  distances.append(euclidean_distance(target,row))

In [107]:
#distances.remove(0.0)
distances.pop(2) # index 2인 원소를 제거

0.0

In [117]:
distances[2] = 99999

In [118]:
np.argmin(distances)

150